In [60]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
import pandas as pd
import timeit
import time
import tensorflow as tf
from joblib import dump, load
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
import pandas as pd
import timeit
import time

import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [61]:
def rbf_svm(train_data, pred_var, data_name):
    t0 = time.time()
    reg = SVR(kernel="rbf")
    reg.fit(train_data[:,:-1], train_data[:,-1])
    t1 = time.time()
    t = t1-t0
    dump(reg, 'results/boosting/models/{}_{}_rbf.joblib'.format(data_name,pred_var))
    np.save("results/boosting/times/{}_{}_rbf_times.npy".format(data_name,pred_var),t)
    return reg, t

def NN(data):
    X_train = data[:,:-1]
    y_train = data[:,-1]
    #create model
    model = Sequential()

    #get number of columns in training data
    n_cols = X_train.shape[1]

    #add model layers
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='relu'))

    model.compile(optimizer='adam', loss='mse')
    early_stopping_monitor = EarlyStopping(patience=10)
    history = model.fit(X_train, y_train, validation_split=0.2, 
                        epochs=100, callbacks=[early_stopping_monitor])

    return model

In [70]:
def boost(d_le1, d_g1, data_name):
    # a/c <= 1 
    data = []
    for row in d_le1:
        if (row[2] > 1.5 and row[2] < 1.65):
            data.append(np.delete(row, 2))
    data = np.array(data)
    reg_M, t = rbf_svm(data, "M", data_name+"_le1")
    M = reg_M.predict(d_le1[:,:2])
    d_le1[:,-1] /= M
    reg_g, t = rbf_svm(d_le1[:,1:], "g", data_name+"_le1")
    
    # a/c > 1 
    data = []
    for row in d_g1:
        if (row[2] > 1.5 and row[2] < 1.65):
            data.append(np.delete(row, 2))
    data = np.array(data)
    reg_M, t = rbf_svm(data, "M", data_name+"_g1")
    M = reg_M.predict(d_g1[:,:2])
    d_g1[:,-1] /= M
    reg_g, t = rbf_svm(d_g1, "g", data_name+"_g1")
    return reg_M, reg_g

In [71]:
d1_g1 = np.load("data/2_RN_eqn_ac_g1.npy")
d1_le1 = np.load("data/2_RN_eqn_ac_le1.npy")

In [72]:
reg_M, reg_g = boost(d1_le1, d1_g1, "d1")

In [74]:
test_data = np.load("data/test/d2_test_g.npy")
M = reg_M.predict(test_data[:,:2])
g = reg_g.predict(test_data[:,:-1])
Mg = M*g

np.mean(Mg - test_data[:,-1])

0.05048539316035211